## Getting data table for date 5/24-5/25:

## importing packages:

In [1]:
import requests #request the html page 
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import sqlalchemy as sqla

## Using BeautifulSoup parse online tables:

In [4]:
url = 'https://www.worldometers.info/coronavirus/#countries' ###
scraping_page = requests.get(url)
scrapinghtml = scraping_page.text
scrapingsoup = BeautifulSoup(scrapinghtml,'lxml')
scrapingsoup


<!DOCTYPE html>
<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]--><!--[if IE 9]> <html lang="en" class="ie9"> <![endif]--><!--[if !IE]><!--><html lang="en">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>COVID Live - Coronavirus Statistics - Worldometer</title>
<meta content="Live statistics and coronavirus news tracking the number of confirmed cases, recovered patients, tests, and death toll due to the COVID-19 coronavirus from Wuhan, China. Coronavirus counter with new cases, deaths, and number of tests per 1 Million population. Historical data and info. Daily charts, graphs, news and updates" name="description"/>
<link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"

In [19]:
extract_table_yesterday = scrapingsoup.find('table', id= 'main_table_countries_yesterday')
extract_table_data_yesterday = extract_table_yesterday.find_all("tr")

extract_table_yesterday2 = scrapingsoup.find('table', id= 'main_table_countries_yesterday2')
extract_table_data_yesterday2 = extract_table_yesterday2.find_all("tr")
date_list = [extract_table_data_yesterday,extract_table_data_yesterday2]

## Storing all data in list:

In [20]:
deepthinker=[[],[]]
content=[]
index = 0
for date in date_list:
    for i in range(9,len(date)): 
        #if nation_name != None:
            for j in range(len(date[i].find_all("td",href = False))):
                content.append(date[i].find_all("td",href = False)[j].text)
            deepthinker[index].append(content)
            content=[]
    index+=1

## Turning list into dataframe and match column names:

In [22]:
dataframe_yesterday = pd.DataFrame(deepthinker[0])
dataframe_yesterday2 = pd.DataFrame(deepthinker[1])
%run ../code/dataframe_organizer.py
df_525 = dataframe_organizer(dataframe_yesterday)
df_524 = dataframe_organizer(dataframe_yesterday2)
df_525['date'] = ['20210525']*222  #We adding date column to the dataframe.
df_524['date'] = ['20210524']*222  

## Adding each dataframe to SQL database:

In [27]:
sqlite_file='../data/covid.sqlite'
covid_engine = sqla.create_engine('sqlite:///'+sqlite_file)
list_df = [df_525,df_524]
for i in list_df:
    i.to_sql('date_table',con=covid_engine,if_exists='append')   